In [18]:
# Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image
import pickle
from tensorflow.keras.models import load_model

In [20]:
# Function to load and preprocess images
def load_and_preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((128, 128))  # Resize image to your preferred size
    img_array = np.array(img) / 255.0  # Normalize pixel values to between 0 and 1
    return img_array

# Function to load data from directories
def load_data(directory):
    data = []
    labels = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if '.jpg' != file[-4:]: continue
            file_path = os.path.join(root, file)
            label = os.path.basename(root)
            data.append(load_and_preprocess_image(file_path))
            labels.append(label)

    return np.array(data), np.array(labels)

In [21]:
# Load data from the specified folders
train_data_cinder, train_labels_cinder = load_data("cinder")
train_data_stripe, train_labels_stripe = load_data("stripe")
train_data_both, train_labels_both = load_data("both")

print("Data Loading and Processing Completed...")

Data Loading and Processing Completed...


In [22]:
# Combine data from all folders
train_data = np.concatenate([train_data_cinder, train_data_stripe, train_data_both])
train_labels = np.concatenate([train_labels_cinder, train_labels_stripe, train_labels_both])

# Shuffle the data
indices = np.arange(train_data.shape[0])
np.random.shuffle(indices)
train_data = train_data[indices]
train_labels = train_labels[indices]

In [23]:
# Specify Model
# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='softmax')  # 4 output classes: cinder, stripe, both, none
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [24]:
# Convert labels to integers
label_mapping = {'cinder': 0, 'stripe': 1, 'both': 2, 'none': 3}
train_labels_int = np.array([label_mapping[label] for label in train_labels])

# Train the model
model.fit(train_data, train_labels_int, epochs=15, validation_split=0.2)
print("Model has completed training")

Epoch 1/15
4/4 [==============================] - 4s 512ms/step - loss: 1.8909 - accuracy: 0.3737 - val_loss: 1.2303 - val_accuracy: 0.2800
Epoch 2/15
4/4 [==============================] - 2s 425ms/step - loss: 1.2000 - accuracy: 0.3333 - val_loss: 1.0642 - val_accuracy: 0.2800
Epoch 3/15
4/4 [==============================] - 2s 401ms/step - loss: 1.0975 - accuracy: 0.3535 - val_loss: 0.9316 - val_accuracy: 0.6400
Epoch 4/15
4/4 [==============================] - 2s 405ms/step - loss: 1.0240 - accuracy: 0.4949 - val_loss: 1.0221 - val_accuracy: 0.6400
Epoch 5/15
4/4 [==============================] - 2s 396ms/step - loss: 1.0054 - accuracy: 0.4949 - val_loss: 0.8655 - val_accuracy: 0.6400
Epoch 6/15
4/4 [==============================] - 2s 422ms/step - loss: 1.0217 - accuracy: 0.4949 - val_loss: 0.9245 - val_accuracy: 0.4400
Epoch 7/15
4/4 [==============================] - 2s 391ms/step - loss: 0.9618 - accuracy: 0.6364 - val_loss: 0.8788 - val_accuracy: 0.7200
Epoch 8/15
4/4 [====

In [25]:
# Save the model accuracy using pickle
accuracy = model.evaluate(train_data, train_labels_int)[1]
with open('model_accuracy.pkl', 'wb') as f:
    pickle.dump(accuracy, f)

# Print the model accuracy
print(f'Model Accuracy: {accuracy}')

# Save the trained model
model.save('kitty_classifier_model.keras')

4/4 [==============================] - 1s 120ms/step - loss: 0.4275 - accuracy: 0.8306
Model Accuracy: 0.8306451439857483


### Now that the training is complete, we will inference the model...

In [16]:
# Load the trained model
model = load_model('image_classifier_model.keras')  # Adjust the path accordingly

# Function to load and preprocess a single image
def load_and_preprocess_single_image(image_path):
    img = Image.open(image_path).convert("RGB")
    img = img.resize((128, 128))  # Resize image to match the training size
    img_array = np.array(img) / 255.0  # Normalize pixel values to between 0 and 1
    return np.expand_dims(img_array, axis=0)  # Add batch dimension

In [35]:
# Load and preprocess the new image
image_path = 'imagec.jpg'  # Replace with your image file path
new_image = load_and_preprocess_single_image(image_path)

# Perform inference
predictions = model.predict(new_image)

# Decode the prediction
class_labels = ['cinder', 'stripe', 'both', 'none']
predicted_class_index = np.argmax(predictions)
predicted_class = class_labels[predicted_class_index]

# Print the predicted class
print(f'The predicted class for the image is: {predicted_class}')

1/1 [==============================] - 0s 44ms/step
The predicted class for the image is: cinder
